# Alternative HEALPix Columns

Author: Melissa

See also: https://github.com/astronomy-commons/hats/issues/550

To support GAIA's `healpix29` column alongside our column named `_healpix_29`, we added a new catalog property: `hats_col_healpix` and `hats_col_healpix_order`.

Having the healpix column take on different orders is pretty contentious, but ... oh well.

In this notebook, we will show interactions between a catalog with defaults (`_healpix_29` and 29) and a really silly bird (`healpix13` and 13).

In [1]:
import lsdb

In [2]:
normal = lsdb.open_catalog("../../tests/data/small_sky_order1_collection")
print(normal.hc_structure.catalog_info)

  catalog_name small_sky_order1
  catalog_type object
  total_rows 131
  ra_column ra
  dec_column dec
  healpix_column _healpix_29
  healpix_order 29
  npix_suffix .parquet
  skymap_order 1



In [3]:
weirdo = lsdb.open_catalog("../../tests/data/small_sky_healpix13")
print(weirdo.hc_structure.catalog_info)

  catalog_name small_sky_healpix13
  catalog_type object
  total_rows 131
  ra_column ra
  dec_column dec
  healpix_column healpix13
  healpix_order 13
  npix_suffix .parquet
  skymap_order 1



Note here that the "healpix13" column doesn't show up, because we don't show the "index" column in this Dataframe view. I think that's WAI.

In [4]:
weirdo

,id,ra,dec,ra_error,dec_error
npartitions=4,,,,,
"Order: 1, Pixel: 44",int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],int64[pyarrow]
"Order: 1, Pixel: 45",...,...,...,...,...
"Order: 1, Pixel: 46",...,...,...,...,...
"Order: 1, Pixel: 47",...,...,...,...,...


But we DO see the values for `healpix13` in the computed data, as we show the index (complete with its name)

In [5]:
weirdo.compute()

,id,ra,dec,ra_error,dec_error
healpix13,,,,,
747530356,703,286.5,-69.5,0,0
742129567,707,308.5,-69.5,0,0
...,...,...,...,...,...
789568885,825,315.5,-30.5,0,0
789143206,829,314.5,-35.5,0,0


For comparison, here's what it looks like when we've got a `_healpix_29` column:

In [6]:
normal.compute()

,id,ra,dec,ra_error,dec_error
_healpix_29,,,,,
3187422220182231470,707,308.5,-69.5,0,0
3187796123455121090,792,320.5,-69.5,0,0
...,...,...,...,...,...
3400255793565258227,767,314.5,-29.5,0,0
3424180623569024089,743,307.5,-25.5,0,0


In [7]:
everything = lsdb.crossmatch(weirdo, normal, suffixes=["_13", "_29"])
everything.compute()

,id_13,ra_13,dec_13,ra_error_13,dec_error_13,id_29,ra_29,dec_29,ra_error_29,dec_error_29,_dist_arcsec
healpix13,,,,,,,,,,,
747530356,703,286.5,-69.5,0,0,703,286.5,-69.5,0,0,0.0
742129567,707,308.5,-69.5,0,0,707,308.5,-69.5,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
789568885,825,315.5,-30.5,0,0,825,315.5,-30.5,0,0,0.0
789143206,829,314.5,-35.5,0,0,829,314.5,-35.5,0,0,0.0


Since the `healpix13` catalog is on the left side here, we lose the `_healpix_29` value, since it was only the index of the right catalog and not one of the columns. Not as convinced that this is WAI.